### Financial Engineering 1: Hedging Assignment

In [ ]:
# imports
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt

from pyfinance.options import BSM

from scipy.stats import norm

In [ ]:
# import data and print head to get rough idea of what is contained
data = pd.read_csv('./msftSept.csv')
data.head()

In [ ]:
# display underlying stock prices in a chart
fig = plt.figure(figsize=(10,6))
plt.plot(data.index, data['Underlying'])
plt.title(f'MSFT Stock Prices from {data["Date"].min()} to {data["Date"].max()}')
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()

In [ ]:
def prepare_data(csvfilepath):
    # read in data
    data = pd.read_csv(csvfilepath)

    # add time to maturity as a column
    maturity = pd.to_datetime(data['Date'].max())
    data['T'] = maturity - pd.to_datetime(data['Date'])
    data['T'] = data['T'].dt.days

    # Add r column (interest rate) - for the moment arbitrary fixed
    data['r'] = 0.05

    # separate data for call and put options
    common_cols = ['Date', 'Underlying', 'T', 'r']
    data_call = data[common_cols + [col for col in data.columns if col.startswith('C')]]
    data_put = data[common_cols + [col for col in data.columns if col.startswith('P')]]

    # melt strike prices into one column for call
    # first remane columns to be a float value
    columnsC_to_transform = [col for col in data_call.columns if col.startswith('C')]
    new_column_namesC = {col: float(col[1:]) for col in columnsC_to_transform}
    data_call.rename(columns=new_column_namesC, inplace=True)
    # Now we can use melt function so that the strike price becomes a variable
    data_call = data_call.melt(id_vars=['Underlying', 'Date', 'T', 'r'], var_name="E").dropna()
    # We rename the Call option price to Cobs
    data_call.rename({'value':'Cobs'}, axis='columns', inplace=True)


    # melt strike prices into one column for put
    # first rename columns to be a float value
    columnsP_to_transform = [col for col in data_put.columns if col.startswith('P')]
    new_column_namesP = {col: float(col[1:]) for col in columnsP_to_transform}
    data_put.rename(columns=new_column_namesP, inplace=True)
    # Melt function for data_put 
    data_put = data_put.melt(id_vars=['Underlying', 'Date', 'T', 'r'], var_name="E").dropna()
    # We rename the Put option price to Pobs
    data_put.rename({'value':'Pobs'}, axis='columns', inplace=True)

    # # Scaling variables of data_call
    data_call['r'] = data_call['r'] / 100
    data_call['T'] = data_call['T'] / 252
    data_call['Underlying'] = data_call['Underlying'] / 1000
    data_call['E'] = data_call['E'] / 1000
    data_call['Cobs'] = data_call['Cobs'] / 1000
    # volatility call
    data_call['volatility'] = data_call.apply(lambda row:  BSM(kind='call', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=0.1).implied_vol(value = row['Cobs']) if row['T'] != 0 else np.nan, axis = 1)

    # Scaling variables of data_put
    data_put['r'] = data_put['r'] / 100
    data_put['T'] = data_put['T'] / 252
    data_put['Underlying'] = data_put['Underlying'] / 1000
    data_put['E'] = data_put['E'] / 1000
    data_put['Pobs'] = data_put['Pobs'] / 1000
    # volatility put
    data_put['volatility'] = data_put.apply(lambda row:  BSM(kind='put', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=0.1).implied_vol(value = row['Pobs']) if row['T'] != 0 else np.nan, axis = 1)

    # Filtering unwanted volatilities, i.e. volatility smaller than zero and larger than 0.6
    data_call = data_call[(data_call['volatility'] < 0.6) & (data_call['volatility'] > 0.0)]
    data_put = data_put[(data_put['volatility'] < 0.6) & (data_put['volatility'] > 0.0)]

    # Adding new column Delta, Vega and Gamma to data_call
    data_call['Delta'] = data_call.apply(lambda row:  BSM(kind='call', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).delta(), axis = 1)
    data_call['Vega'] = data_call.apply(lambda row:  BSM(kind='call', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).vega(), axis = 1)
    data_call['Gamma'] = data_call.apply(lambda row:  BSM(kind='call', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).gamma(), axis = 1)

    # Adding new column Delta, Vega and Gamma to data_put
    data_put['Delta'] = data_put.apply(lambda row:  BSM(kind='put', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).delta(), axis = 1)
    data_put['Vega'] = data_put.apply(lambda row:  BSM(kind='put', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).vega(), axis = 1)
    data_put['Gamma'] = data_put.apply(lambda row:  BSM(kind='put', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).gamma(), axis = 1)

    return data_call, data_put

def get_option_portfolio_values(portfolio, start_date, end_date, all_options, call_options, put_options):
    portfolio_values = {date: np.NaN for date in all_options.loc[(all_options['Date'] >= start_date) & (all_options['Date'] <= end_date), 'Date']}

    for option, quantity, buy_sell in portfolio:
        option_type = option[0]
        option_strike = float(option[1:])

        if(option_type == 'C'):
            options_data = call_options
            option_price_column = 'Cobs'
        else:
            options_data = put_options
            option_price_column = 'Pobs'

        for index, row in options_data[(options_data['Date'] >= start_date) & (options_data['Date'] <= end_date) & (options_data['E'] == (option_strike/1000))].iterrows():              
            if (np.isnan(portfolio_values[row['Date']])):
                portfolio_values[row['Date']] = 0
            
            if buy_sell == 'buy':
                portfolio_values[row['Date']] += (row[option_price_column] * quantity)
            else:
                portfolio_values[row['Date']] -= (row[option_price_column] * quantity)

    sorted_portfolio_values = dict(sorted(portfolio_values.items()))
                
    return np.array(list(sorted_portfolio_values.values()))

def get_values(ticker, start_date, end_date, all_options, call_options, put_options):
    values = {date: np.NaN for date in all_options.loc[(all_options['Date'] >= start_date) & (all_options['Date'] <= end_date), 'Date']}

    options_data = all_options

    if(ticker == 'Underlying'):
        for index, row in options_data[(options_data['Date'] >= start_date) & (options_data['Date'] <= end_date)].iterrows():
            values[row['Date']] = row['Underlying']
        sorted_values = dict(sorted(values.items()))
        return np.array(list(sorted_values.values()))

    if(ticker[0] == 'C'):
        options_data = call_options
        option_price_column = 'Cobs'
    elif(ticker[0] == 'P'):
        options_data = put_options
        option_price_column = 'Pobs'

    option_strike = float(ticker[1:])

    for index, row in options_data[(options_data['Date'] >= start_date) & (options_data['Date'] <= end_date) & (options_data['E'] == (option_strike/1000))].iterrows():       
        values[row['Date']] = row[option_price_column]

    sorted_values = dict(sorted(values.items()))
        
    return np.array(list(sorted_values.values()))

def get_delta_values(portfolio, start_date, end_date, all_options, call_options, put_options):
    delta_values = {date: np.NaN for date in all_options.loc[(all_options['Date'] >= start_date) & (all_options['Date'] <= end_date), 'Date']}

    for option, quantity, buy_sell in portfolio:
        option_type = option[0]
        option_strike = float(option[1:])

        if(option_type == 'C'):
            options_data = call_options
        else:
            options_data = put_options

        for index, row in options_data[(options_data['Date'] >= start_date) & (options_data['Date'] <= end_date) & (options_data['E'] == (option_strike/1000))].iterrows():         
            if (np.isnan(delta_values[row['Date']])):
                delta_values[row['Date']] = 0
            
            if buy_sell == 'buy':
                delta_values[row['Date']] += (row['Delta'] * quantity)
            else:
                delta_values[row['Date']] -= (row['Delta'] * quantity)

    sorted_delta_values = dict(sorted(delta_values.items()))

    return np.array(list(sorted_delta_values.values()))

def get_vega_values(portfolio, start_date, end_date, all_options, call_options, put_options):
    vega_values = {date: np.NaN for date in all_options.loc[(all_options['Date'] >= start_date) & (all_options['Date'] <= end_date), 'Date']}

    for option, quantity, buy_sell in portfolio:
        option_type = option[0]
        option_strike = float(option[1:])

        if(option_type == 'C'):
            options_data = call_options
        else:
            options_data = put_options

        for index, row in options_data[(options_data['Date'] >= start_date) & (options_data['Date'] <= end_date) & (options_data['E'] == (option_strike/1000))].iterrows():
            if np.isnan(row['Vega']):
                vega_values[row['Date']] = np.NaN
            
            if (np.isnan(vega_values[row['Date']])):
                vega_values[row['Date']] = 0
            
            if buy_sell == 'buy':
                vega_values[row['Date']] += (row['Vega'] * quantity)
            else:
                vega_values[row['Date']] -= (row['Vega'] * quantity)

    sorted_vega_values = dict(sorted(vega_values.items()))

    return np.array(list(sorted_vega_values.values()))

def delta_hedge(hedging_frequency, portfolio_values, underlying_values, delta_values):
    if hedging_frequency <= 0 or hedging_frequency >= len(delta_values):
        return False
    
    portfolio_value = portfolio_values[0]
    underlying_value = underlying_values[0]
    delta = delta_values[0]
    accuracies = []

    transaction_cost = 0

    missing_data = 0

    for i in range(hedging_frequency, len(delta_values), hedging_frequency):
        if(not(np.isnan(portfolio_values[i]) or np.isnan(portfolio_value) or np.isnan(underlying_value) or np.isnan(underlying_values[i]) or np.isnan(delta))):
            accuracy = (portfolio_values[i] - portfolio_value) - (underlying_values[i] - underlying_value) * delta
            accuracies.append(accuracy)
            portfolio_value = portfolio_values[i]
            transaction_cost += abs(underlying_values[i] - underlying_value) * 0.05
            underlying_value = underlying_values[i]
            delta = delta_values[i]
        else:
            missing_data += 1

    print(f'{missing_data} missing data were detected during the hedging.')

    mean_squared_error = sum([num**2 for num in accuracies]) / len(accuracies)

    return {
        'accuracies': accuracies,
        'mean_squered_error': mean_squared_error,
        'transaction_cost': transaction_cost
    }

def option_portfolio_delta_hedging(portfolio, start_date, end_date, hedging_frequency, all_options, call_options, put_options):
    # portfolio example: [['C250', 2, 'buy'], ['P400', 1, 'sell']]

    portfolio_values = get_option_portfolio_values(portfolio, start_date, end_date, all_options, call_options, put_options)
    underlying_values = get_values('Underlying', start_date, end_date, all_options, call_options, put_options)
    delta_values = get_delta_values(portfolio, start_date, end_date, all_options, call_options, put_options)

    # print(portfolio_values)
    # print(underlying_values)
    # print(delta_values)

    # print(len(portfolio_values))
    # print(len(underlying_values))
    # print(len(delta_values))

    hedging_results = delta_hedge(hedging_frequency, portfolio_values, underlying_values, delta_values)
    return hedging_results

def solve_2_by_2_linear_system(equation_one, equation_two):
    # equations of the form ax + by = c, dx + ey = f
    a, b, c = equation_one
    d, e, f = equation_two

    x = (b * f - e * c) / (b * d - a * e)
    y = (c - a * x) / b

    return (x, y)

def delta_vega_hedge(hedging_frequency, portfolio_values, portfolio_delta_values, portfolio_vega_values, sec_one_values, sec_one_delta_values, 
                        sec_one_vega_values, sec_two_values, sec_two_delta_values, sec_two_vega_values):

    portfolio_value = portfolio_values[0]
    portfolio_delta= portfolio_delta_values[0]
    portfolio_vega = portfolio_vega_values[0]
    sec_one = sec_one_values[0]
    sec_one_delta = sec_one_delta_values[0]
    sec_one_vega = sec_one_vega_values[0]
    sec_two = sec_two_values[0]
    sec_two_delta = sec_two_delta_values[0]
    sec_two_vega = sec_two_vega_values[0]
    sec_one_amount, sec_two_amount = solve_2_by_2_linear_system([sec_one_delta, sec_two_delta, -portfolio_delta], 
                                                                    [sec_one_vega, sec_two_vega, -portfolio_vega])
    accuracies = []

    transaction_cost = 0

    for i in range(hedging_frequency, len(sec_two_delta_values), hedging_frequency):
        change_in_portfolio = portfolio_values[i] - portfolio_value
        change_in_replication = sec_one_amount * (sec_one_values[i] - sec_one) + sec_two_amount * (sec_two_values[i] - sec_two)
        accuracy = change_in_portfolio - change_in_replication
        accuracies.append(accuracy)
        transaction_cost += change_in_replication * 0.05
        portfolio_value = portfolio_values[i]
        portfolio_delta= portfolio_delta_values[i]
        portfolio_vega = portfolio_vega_values[i]
        sec_one = sec_one_values[i]
        sec_one_delta = sec_one_delta_values[i]
        sec_one_vega = sec_one_vega_values[i]
        sec_two = sec_two_values[i]
        sec_two_delta = sec_two_delta_values[i]
        sec_two_vega = sec_two_vega_values[i]
        sec_one_amount, sec_two_amount = solve_2_by_2_linear_system([sec_one_delta, sec_two_delta, -portfolio_delta], 
                                                                        [sec_one_vega, sec_two_vega, -portfolio_vega])

    mean_squared_error = sum([num**2 for num in accuracies]) / len(accuracies)

    return {
        'accuracies': accuracies,
        'mean_squered_error': mean_squared_error,
        'transaction_cost': transaction_cost
    }

def option_portfolio_delta_vega_hedging(portfolio, security_one, security_two, start_date, end_date, hedging_frequency, all_options, call_options, put_options, sec1_call_data, sec2_call_data):
    # portfolio example: [['C250', 2, 'buy'], ['P400', 1, 'sell']]

    portfolio_values = get_option_portfolio_values(portfolio, start_date, end_date, all_options, call_options, put_options)
    portfolio_delta_values = get_delta_values(portfolio, start_date, end_date, all_options, call_options, put_options)
    portfolio_vega_values = get_vega_values(portfolio, start_date, end_date, all_options, call_options, put_options)
    sec_one_values = get_values(security_one, start_date, end_date, all_options, sec1_call_data, put_options)
    sec_one_delta_values = get_delta_values([[security_one, 1, 'buy']], start_date, end_date, all_options, sec1_call_data, put_options)
    sec_one_vega_values = get_vega_values([[security_one, 1, 'buy']], start_date, end_date, all_options, sec1_call_data, put_options)
    sec_two_values = get_values(security_two, start_date, end_date, all_options, sec2_call_data, put_options)
    sec_two_delta_values = get_delta_values([[security_two, 1, 'buy']], start_date, end_date, all_options, sec2_call_data, put_options)
    sec_two_vega_values = get_vega_values([[security_two, 1, 'buy']], start_date, end_date, all_options, sec2_call_data, put_options)
    
    hedging_results = delta_vega_hedge(hedging_frequency, portfolio_values, portfolio_delta_values, portfolio_vega_values, sec_one_values, 
                                            sec_one_delta_values, sec_one_vega_values, sec_two_values, sec_two_delta_values, sec_two_vega_values)
    return hedging_results

In [ ]:
data = pd.read_csv('./msftOct.csv')
oct_call_data, oct_put_data = prepare_data('./msftOct.csv')
sept_call_data, sept_put_data = prepare_data('./msftSept.csv')
nov_call_data, nov_put_data = prepare_data('./fe2.csv')


In [ ]:
# Update T, volatily, delta, gamma and vega based on expiration date
def update_greeks_and_T_based_on_expiration_date(expiration_date, call_options, put_options):
    call_options['T'] = pd.to_datetime(expiration_date) - pd.to_datetime(call_options['Date'])
    call_options['T'] = call_options['T'].dt.days
    call_options['T'] = call_options['T'] / 252

    put_options['T'] = pd.to_datetime(expiration_date) - pd.to_datetime(put_options['Date'])
    put_options['T'] = put_options['T'].dt.days
    put_options['T'] = put_options['T'] / 252

    call_options['volatility'] = call_options.apply(lambda row:  BSM(kind='call', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=0.1).implied_vol(value = row['Cobs']) if row['T'] != 0 else np.nan, axis = 1)
    call_options['Delta'] = call_options.apply(lambda row:  BSM(kind='call', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).delta(), axis = 1)
    call_options['Vega'] = call_options.apply(lambda row:  BSM(kind='call', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).vega(), axis = 1)
    call_options['Gamma'] = call_options.apply(lambda row:  BSM(kind='call', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).gamma(), axis = 1)

    put_options['volatility'] = put_options.apply(lambda row:  BSM(kind='put', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=0.1).implied_vol(value = row['Pobs']) if row['T'] != 0 else np.nan, axis = 1)
    put_options['Delta'] = put_options.apply(lambda row:  BSM(kind='put', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).delta(), axis = 1)
    put_options['Vega'] = put_options.apply(lambda row:  BSM(kind='put', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).vega(), axis = 1)
    put_options['Gamma'] = put_options.apply(lambda row:  BSM(kind='put', S0=row['Underlying'], K=row['E'], T= row['T'], r=row['r'], sigma=row['volatility']).gamma(), axis = 1)


In [ ]:
def get_atm_call_price(date, call_options):
    # Find the call option price with delta value that is closest to 0.5 on the start date, to get the ATM option
    filtered_rows = call_options[pd.to_datetime(oct_call_data['Date']) == date]
    atm_call = filtered_rows.loc[(filtered_rows['Delta'] - 0.5).abs().idxmin()]
    strike = atm_call['E']
    return strike*1000

In [ ]:
def get_atm_put_price(date, put_options):
    # Find the call option price with delta value that is closest to 0.5 on the start date, to get the ATM option
    filtered_rows = put_options[pd.to_datetime(oct_put_data['Date']) == date]
    atm_call = filtered_rows.loc[(filtered_rows['Delta'] - 0.5).abs().idxmin()]
    strike = atm_call['E']
    return strike*1000

In [ ]:
# Impact of changing Hedging frequency on the accuracy of Delta Hedging 
# Fixed variables: Time to maturity = 45 days, ATM call price

def testing_multiple_hedging_frequencies(all_options, call_options, put_options, sec1_call_data, sec2_call_data):
    mean_errors_delta_hedging = []
    mean_errors_delta_vega_hedging = []
    mean_transaction_cost_delta_hedging = []
    mean_transaction_cost_delta_vega_hedging = []
    # Testing hedging frequency 1 day, 2 days ... 10 days
    iteration_hedging_frequencies = np.arange(1, 11)

    for iteration_hedging_frequency in iteration_hedging_frequencies:
        errors_delta_hedging = [] 
        errors_delta_vega_hedging = [] 
        transaction_cost_delta_hedging = [] 
        transaction_cost_delta_vega_hedging = [] 

        # 10 different expirations date for statistical results
        for iteration_expiration_date in range(10):
            expiration_date = all_options.at[all_options[all_options['Date'] == '2023-10-20'].index[0] - iteration_expiration_date*2, 'Date']

            # print(expiration_date)
            update_greeks_and_T_based_on_expiration_date(expiration_date, call_options, put_options)

            start_date = all_options.at[all_options[all_options['Date'] == expiration_date].index[0] - 45, 'Date']
            # print(start_date)

            atm_call_price = get_atm_call_price(start_date, call_options)

            # print('C'+str(int(atm_call_price)))

            result_delta_hedging = option_portfolio_delta_hedging([['C'+str(int(atm_call_price)), 1, 'buy']], start_date, expiration_date, iteration_hedging_frequency, all_options, call_options, put_options)
            errors_delta_hedging.append(result_delta_hedging['mean_squered_error'])
            transaction_cost_delta_hedging.append(result_delta_hedging['transaction_cost'])

            result_delta_vega_hedging = option_portfolio_delta_vega_hedging([['C'+str(int(atm_call_price)), 1, 'buy']], 'C250', 'C300', start_date, expiration_date, iteration_hedging_frequency, all_options, call_options, put_options, sec1_call_data, sec2_call_data)
            errors_delta_vega_hedging.append(result_delta_vega_hedging['mean_squered_error'])
            transaction_cost_delta_vega_hedging.append(result_delta_vega_hedging['transaction_cost'])


        mean_errors_delta_hedging.append(np.mean(errors_delta_hedging))
        mean_errors_delta_vega_hedging.append(np.mean(errors_delta_vega_hedging))
        mean_transaction_cost_delta_hedging.append(np.mean(transaction_cost_delta_hedging))
        mean_transaction_cost_delta_vega_hedging.append(np.mean(transaction_cost_delta_vega_hedging))

    # Save data to files
    delta_performance_data_multiple_hedging_frequencies = pd.DataFrame({'Hedging Frequency': iteration_hedging_frequencies, 'Mean Squared Error Delta Hedging': mean_errors_delta_hedging})
    delta_vega_performance_data_multiple_hedging_frequencies = pd.DataFrame({'Hedging Frequency': iteration_hedging_frequencies, 'Mean Squared Error Delta-Vega Hedging': mean_errors_delta_vega_hedging})

    delta_performance_data_multiple_hedging_frequencies.to_csv('delta_performance_multiple_hedging_frequencies.csv', index=False)
    delta_vega_performance_data_multiple_hedging_frequencies.to_csv('delta_vega_performance_multiple_hedging_frequencies.csv', index=False)

    # Plotting
    plt.plot(iteration_hedging_frequencies, mean_errors_delta_hedging, marker='o')
    plt.xlabel('Hedging Frequency (Days)')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Error vs. Hedging Frequency (Delta Hedging)')
    plt.show()

    plt.plot(iteration_hedging_frequencies, mean_errors_delta_vega_hedging, marker='o')
    plt.xlabel('Hedging Frequency (Days)')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Error vs. Hedging Frequency (Delta Vega Hedging)')
    plt.show()

    plt.plot(iteration_hedging_frequencies, mean_errors_delta_hedging, label='Delta Hedging', marker='o')
    plt.plot(iteration_hedging_frequencies, mean_errors_delta_vega_hedging, label='Delta-Vega Hedging', marker='o')
    plt.xlabel('Hedging Frequency (Days)')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Error vs. Hedging Frequency (Comparing Delta and Delta-Vega Hedging Strategies)')
    plt.legend()
    plt.show()

    plt.plot(iteration_hedging_frequencies, mean_transaction_cost_delta_hedging, label='Delta Hedging', marker='o')
    plt.plot(iteration_hedging_frequencies, mean_transaction_cost_delta_vega_hedging, label='Delta-Vega Hedging', marker='o')
    plt.xlabel('Hedging Frequency (Days)')
    plt.ylabel('Transaction cost')
    plt.title('Transaction cost vs. Hedging Frequency (Comparing Delta and Delta-Vega Hedging Strategies)')
    plt.legend()
    plt.show()


In [ ]:
testing_multiple_hedging_frequencies(data, oct_call_data, oct_put_data, nov_call_data, nov_call_data)

In [ ]:
# Impact of changing time to maturity on the accuracy of Delta Hedging 
# Fixed variables: Hedging every day, ATM call price

def testing_multiple_time_to_maturity(all_options, call_options, put_options, sec1_call_data, sec2_call_data):
    mean_errors_delta_hedging = []
    mean_errors_delta_vega_hedging = []
    mean_transaction_cost_delta_hedging = [] 
    mean_transaction_cost_delta_vega_hedging = [] 
    # Testing time to maturity 10, 20, ..., 60
    iteration_time_to_maturities = np.arange(10, 41, 5)

    for iteration_time_to_maturity in iteration_time_to_maturities:
        errors_delta_hedging = [] 
        errors_delta_vega_hedging = [] 
        transaction_cost_delta_hedging = [] 
        transaction_cost_delta_vega_hedging = [] 
        # 10 different expirations date for statistical results
        for iteration_expiration_date in range(10):
            expiration_date = all_options.at[all_options[all_options['Date'] == '2023-10-20'].index[0] - iteration_expiration_date*2, 'Date']
    
            update_greeks_and_T_based_on_expiration_date(expiration_date, call_options, put_options)

            start_date = all_options.at[all_options[all_options['Date'] == expiration_date].index[0] - iteration_time_to_maturity, 'Date']

            atm_call_price = get_atm_call_price(start_date, oct_call_data)

            result_delta_hedging = option_portfolio_delta_hedging([['C'+str(int(atm_call_price)), 1, 'buy']], start_date, expiration_date, 1, all_options, call_options, put_options)
            errors_delta_hedging.append(result_delta_hedging['mean_squered_error'])
            transaction_cost_delta_hedging.append(result_delta_hedging['transaction_cost'])

            result_delta_vega_hedging = option_portfolio_delta_vega_hedging([['C'+str(int(atm_call_price)), 1, 'buy']], 'C250', 'C300',start_date, expiration_date, 1, all_options, call_options, put_options, sec1_call_data, sec2_call_data)
            errors_delta_vega_hedging.append(result_delta_vega_hedging['mean_squered_error'])
            transaction_cost_delta_vega_hedging.append(result_delta_vega_hedging['transaction_cost'])

        mean_errors_delta_hedging.append(np.mean(errors_delta_hedging))
        mean_errors_delta_vega_hedging.append(np.mean(errors_delta_vega_hedging))
        mean_transaction_cost_delta_hedging.append(np.mean(transaction_cost_delta_hedging))
        mean_transaction_cost_delta_vega_hedging.append(np.mean(transaction_cost_delta_vega_hedging))

    # Save data to files
    delta_performance_data_multiple_time_to_maturity = pd.DataFrame({'Time to Maturity': iteration_time_to_maturities, 'Mean Squared Error Delta Hedging': mean_errors_delta_hedging})
    delta_vega_performance_data_multiple_time_to_maturity = pd.DataFrame({'Time to Maturity': iteration_time_to_maturities, 'Mean Squared Error Delta-Vega Hedging': mean_errors_delta_vega_hedging})

    delta_performance_data_multiple_time_to_maturity.to_csv('delta_performance_multiple_time_to_maturity.csv', index=False)
    delta_vega_performance_data_multiple_time_to_maturity.to_csv('delta_vega_performance_multiple_time_to_maturity.csv', index=False)

    # Plotting
    plt.plot(iteration_time_to_maturities, mean_errors_delta_hedging, marker='o')
    plt.xlabel('Time to maturity (Days)')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Error vs. Time to maturity (Delta Hedging)')
    plt.show()

    plt.plot(iteration_time_to_maturities, mean_errors_delta_hedging, marker='o')
    plt.xlabel('Time to maturity (Days)')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Error vs. Time to maturity (Delta-Vega Hedging)')
    plt.show()

    plt.plot(iteration_time_to_maturities, mean_errors_delta_hedging, label='Delta Hedging', marker='o')
    plt.plot(iteration_time_to_maturities, mean_errors_delta_vega_hedging, label='Delta-Vega Hedging', marker='o')
    plt.xlabel('Time to Maturity (Days)')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Error vs. Time to Maturity (Comparing Delta and Delta-Vega Hedging Strategies)')
    plt.legend()
    plt.show()

    plt.plot(iteration_time_to_maturities, mean_transaction_cost_delta_hedging, label='Delta Hedging', marker='o')
    plt.plot(iteration_time_to_maturities, mean_transaction_cost_delta_vega_hedging, label='Delta-Vega Hedging', marker='o')
    plt.xlabel('Time to maturity (Days)')
    plt.ylabel('Transaction cost')
    plt.title('Transaction cost vs. Time to Maturity (Comparing Delta and Delta-Vega Hedging Strategies)')
    plt.legend()
    plt.show()


In [ ]:
testing_multiple_time_to_maturity(data, oct_call_data, oct_put_data, nov_call_data, nov_call_data)

In [ ]:
# Impact of changing Strike prices on the accuracy of Delta Hedging 
# Fixed variables: Time to maturity = 45 days, Hedging every day

def testing_multiple_call_prices(all_options, call_options, put_options, sec1_call_data, sec2_call_data):
    mean_errors_delta_hedging = np.zeros(5)
    mean_errors_delta_vega_hedging = np.zeros(5)
    mean_transaction_cost_delta_hedging = np.zeros(5)
    mean_transaction_cost_delta_vega_hedging = np.zeros(5)

    for iteration_expiration_date in range(10):
        expiration_date = all_options.at[all_options[all_options['Date'] == '2023-10-20'].index[0] - iteration_expiration_date*2, 'Date']
        update_greeks_and_T_based_on_expiration_date(expiration_date, call_options, put_options)
        start_date = all_options.at[all_options[all_options['Date'] == expiration_date].index[0] - 45, 'Date']

        atm_call_price = get_atm_call_price(start_date, oct_call_data)

        prices = [atm_call_price-10, atm_call_price-5, atm_call_price, atm_call_price+5, atm_call_price+10]
        counter = 0
        for price in prices:
            result_delta_hedging = option_portfolio_delta_hedging([['C'+str(int(price)), 1, 'buy']], start_date, expiration_date, 1, all_options, call_options, put_options)
            mean_errors_delta_hedging[counter] += result_delta_hedging['mean_squered_error']
            mean_transaction_cost_delta_hedging[counter] += result_delta_hedging['transaction_cost']

            result_delta_vega_hedging = option_portfolio_delta_vega_hedging([['C'+str(int(price)), 1, 'buy']], 'C250', 'C300', start_date, expiration_date, 1, all_options, call_options, put_options, sec1_call_data, sec2_call_data)
            mean_errors_delta_vega_hedging[counter] += result_delta_vega_hedging['mean_squered_error']
            mean_transaction_cost_delta_vega_hedging[counter] += result_delta_vega_hedging['transaction_cost']

            counter += 1

    mean_errors_delta_hedging = (mean_errors_delta_hedging / 10.0)
    mean_errors_delta_vega_hedging = (mean_errors_delta_vega_hedging / 10.0)
    mean_transaction_cost_delta_hedging = (mean_transaction_cost_delta_hedging / 10.0)
    mean_transaction_cost_delta_vega_hedging = (mean_transaction_cost_delta_vega_hedging / 10.0)

    legend = ['ATM - 10', 'ATM - 5', 'ATM', 'ATM + 5', 'ATM + 10']

    # Save data to files
    delta_performance_data = pd.DataFrame({'Call prices': legend, 'Mean Squared Error Delta Hedging': mean_errors_delta_hedging})
    delta_vega_performance_data = pd.DataFrame({'Call prices': legend, 'Mean Squared Error Delta-Vega Hedging': mean_errors_delta_vega_hedging})

    delta_performance_data.to_csv('delta_performance_multiple_call_prices.csv', index=False)
    delta_vega_performance_data.to_csv('delta_vega_performance_multiple_call_prices.csv', index=False)

    # Plotting
    plt.plot(legend, mean_errors_delta_hedging, marker='o')
    plt.xlabel('Call price')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Error vs. Call price (Delta Hedging)')
    plt.show()

    plt.plot(legend, mean_errors_delta_vega_hedging, marker='o')
    plt.xlabel('Call price')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Error vs. Call price (Delta-Vega Hedging)')
    plt.show()

    plt.plot(legend, mean_errors_delta_hedging, label='Delta Hedging', marker='o')
    plt.plot(legend, mean_errors_delta_vega_hedging, label='Delta-Vega Hedging', marker='o')
    plt.xlabel('Call price')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Error vs. Call price (Comparing Delta and Delta-Vega Hedging Strategies)')
    plt.legend()
    plt.show()

    plt.plot(legend, mean_transaction_cost_delta_hedging, label='Delta Hedging', marker='o')
    plt.plot(legend, mean_transaction_cost_delta_vega_hedging, label='Delta-Vega Hedging', marker='o')
    plt.xlabel('Call price')
    plt.ylabel('Transaction cost')
    plt.title('Transaction cost vs. Call price (Comparing Delta and Delta-Vega Hedging Strategies)')
    plt.legend()
    plt.show()


In [ ]:
testing_multiple_call_prices(data, oct_call_data, oct_put_data, nov_call_data, nov_call_data)